<a href="https://colab.research.google.com/github/s-miramontes/News_Filter/blob/emma/Pilot/Data%20Cleanse%20and%20Clustering%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering News Headlines

In this notebook we begin my importing data to analyze its contents and be able to determine the best clustering algorithm to determine the articles that are mostly related to each other.

We start by importing some dependencies

In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import re
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn

In [0]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub

import os

import heapq
import operator

## Importing Data from Local File System

Datasets are located here: https://www.kaggle.com/snapcrack/all-the-news/version/4#articles3.csv

Proceed to download all three 'csv' files, and store them in a 'data' directory at the location of your choice.


*Note: It might be a better idea to have a way to import the data from the cloud (google drive) rather than from the user's local file system. Let's leave it like this for now, mainly bringing this up for easy reproducibility.*

Instead, we can mount google drive and import the datasets from an existing file in the cloud.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

For every file available we have a shared link:

CSV File: 'Articles 3':
https://drive.google.com/a/berkeley.edu/file/d/1BDhi5FuIvJ_6jiIdIFXRjT0w6tgfNk10/view?usp=sharing

CSV File: 'Articles 2':
https://drive.google.com/a/berkeley.edu/file/d/1qpoRkKEOnxYg_12e0RdeRi-JWUGWNFP3/view?usp=sharing

CSV File: 'Articles 1':
https://drive.google.com/a/berkeley.edu/file/d/1c9NbOIx7M6PgyJKxR9E7HgzuJlxGIaXU/view?usp=sharing


For each of the links above, we utiliz the File ID provided:

- Articles 1 File ID: '1BDhi5FuIvJ_6jiIdIFXRjT0w6tgfNk10'
- Articles 2 File ID: '1qpoRkKEOnxYg_12e0RdeRi-JWUGWNFP3'
- Articles 3 File ID: '1c9NbOIx7M6PgyJKxR9E7HgzuJlxGIaXU'


In [0]:
downloaded3 = drive.CreateFile({'id':"1BDhi5FuIvJ_6jiIdIFXRjT0w6tgfNk10"})   # replace the id with id of file you want to access
downloaded3.GetContentFile('articles3.csv.zip')        # replace the file name with your file

downloaded2 = drive.CreateFile({'id': "1qpoRkKEOnxYg_12e0RdeRi-JWUGWNFP3"})
downloaded2.GetContentFile('articles2.csv.zip')

downloaded1 = drive.CreateFile({'id': "1c9NbOIx7M6PgyJKxR9E7HgzuJlxGIaXU"})
downloaded1.GetContentFile('articles1.csv.zip')

In [0]:
articles_3 = pd.read_csv('articles3.csv.zip', compression='zip')
articles_2 = pd.read_csv('articles2.csv.zip', compression='zip')
articles_1 = pd.read_csv('articles1.csv.zip', compression='zip')

In [8]:
articles_3.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,103459,151908,Alton Sterling’s son: ’Everyone needs to prote...,Guardian,Jessica Glenza,2016-07-13,2016.0,7.0,https://www.theguardian.com/us-news/2016/jul/1...,The son of a Louisiana man whose father was sh...
1,103460,151909,Shakespeare’s first four folios sell at auctio...,Guardian,NaN,2016-05-25,2016.0,5.0,https://www.theguardian.com/culture/2016/may/2...,Copies of William Shakespeare’s first four boo...
2,103461,151910,My grandmother’s death saved me from a life of...,Guardian,Robert Pendry,2016-10-31,2016.0,10.0,https://www.theguardian.com/commentisfree/2016...,"Debt: $20, 000, Source: College, credit cards,..."
3,103462,151911,I feared my life lacked meaning. Cancer pushed...,Guardian,Bradford Frost,2016-11-26,2016.0,11.0,https://www.theguardian.com/commentisfree/2016...,"It was late. I was drunk, nearing my 35th birt..."
4,103463,151912,Texas man serving life sentence innocent of do...,Guardian,NaN,2016-08-20,2016.0,8.0,https://www.theguardian.com/us-news/2016/aug/2...,A central Texas man serving a life sentence fo...


In [9]:
articles_2.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,53293,73471,Patriots Day Is Best When It Digs Past the Her...,Atlantic,David Sims,2017-01-11,2017.0,1.0,NaN,"Patriots Day, Peter Berg’s new thriller that r..."
1,53294,73472,A Break in the Search for the Origin of Comple...,Atlantic,Ed Yong,2017-01-11,2017.0,1.0,NaN,"In Norse mythology, humans and our world were ..."
2,53295,73474,Obama’s Ingenious Mention of Atticus Finch,Atlantic,Spencer Kornhaber,2017-01-11,2017.0,1.0,NaN,“If our democracy is to work in this increasin...
3,53296,73475,"Donald Trump Meets, and Assails, the Press",Atlantic,David A. Graham,2017-01-11,2017.0,1.0,NaN,Updated on January 11 at 5:05 p. m. In his fir...
4,53297,73476,Trump: ’I Think’ Hacking Was Russian,Atlantic,Kaveh Waddell,2017-01-11,2017.0,1.0,NaN,Updated at 12:25 p. m. After months of equivoc...


In [10]:
articles_1.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [11]:
set(articles_3.publication)

{'Guardian', 'NPR', 'Reuters', 'Vox', 'Washington Post'}

In [12]:
set(articles_2.publication)

{'Atlantic',
 'Buzzfeed News',
 'Fox News',
 'Guardian',
 'National Review',
 'New York Post',
 'Talking Points Memo'}

In [13]:
set(articles_1.publication)

{'Atlantic', 'Breitbart', 'Business Insider', 'CNN', 'New York Times'}

In [0]:
full_data = pd.concat([articles_1, articles_2, articles_3], ignore_index=True)

In [15]:
full_data

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...,...,...,...,...,...,...
142565,146028,218078,An eavesdropping Uber driver saved his 16-year...,Washington Post,Avi Selk,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Uber driver Keith Avila picked up a p...
142566,146029,218079,Plane carrying six people returning from a Cav...,Washington Post,Sarah Larimer,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Crews on Friday continued to search L...
142567,146030,218080,After helping a fraction of homeowners expecte...,Washington Post,Renae Merle,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,When the Obama administration announced a...
142568,146031,218081,"Yes, this is real: Michigan just banned bannin...",Washington Post,Chelsea Harvey,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,This story has been updated. A new law in...


In [0]:
# remove duplicates
full_data = full_data.drop_duplicates(subset=['title', 'publication', 'author', 'date'])

In [0]:
# remove missing titles 
full_data = full_data.dropna(subset=['title'])

In [18]:
# sample from full_data (set seed to 5)
small_data = full_data.sample(n=10000, random_state=5).reset_index()
set(small_data.publication)

{'Atlantic',
 'Breitbart',
 'Business Insider',
 'Buzzfeed News',
 'CNN',
 'Fox News',
 'Guardian',
 'NPR',
 'National Review',
 'New York Post',
 'New York Times',
 'Reuters',
 'Talking Points Memo',
 'Vox',
 'Washington Post'}

In [19]:
small_data

,index,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,74496,77946,118473,"Chaos in the Family, Chaos in the State: The W...",National Review,Kevin D. Williamson,2016-03-17,2016.0,3.0,http://www.nationalreview.com/article/432876/d...,Michael Brendan Dougherty is bitter. I think t...
1,71184,74592,113594,US Civil Rights Commission Will Observe Stand...,Buzzfeed News,Nidhi Subbaraman,2016-12-08,2016.0,12.0,https://web.archive.org/web/20161208153906/htt...,WASHINGTON — The US Commission on Civil Ri...
2,120205,123668,184574,"Venezuela hunts rogue helicopter attackers, Ma...",Reuters,Andrew Cawthorne and Victoria Ramirez,2017-06-29,2017.0,6.0,http://www.reuters.com/article/us-venezuela-po...,The Venezuelan government hunted on Wednesday...
3,128977,132440,199665,Fruit juice isn’t much better for you than sod...,Vox,Julia Belluz,2016/3/25,2016.0,3.0,http://www.vox.com/2016/3/25/11305614/soda-jui...,One of the biggest public health wins of rece...
4,134837,138300,208223,Sessions won’t testify at congressional budget...,Washington Post,Sari Horwitz,2017-06-10,2017.0,6.0,https://web.archive.org/web/20170611000758/htt...,"Attorney General Jeff Sessions, who had agree..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,137105,140568,211140,Patient secretly recorded doctors as they oper...,Washington Post,Yanan Wang,2016-04-07,2016.0,4.0,https://web.archive.org/web/20160408000201/htt...,"Last summer, Ethel Easter wanted nothin..."
9996,60293,63612,86308,Fox News Poll: Clinton edges Trump by two poin...,Fox News,Dana Blanton,2016-10-07,2016.0,10.0,https://web.archive.org/web/20161008002456/htt...,Third party candidates Gary Johnson (6 percent...
9997,55079,58389,80070,The Atlantic Politics & Policy Daily: Trump’s...,Atlantic,Elaine Godfrey,2016-08-31,2016.0,8.0,NaN,For us to continue writing great stori...
9998,110248,113711,168509,A Son In Chains. A Depressed Mom. Here’s What ...,NPR,Nurith Aizenman,2016-04-15,2016.0,4.0,http://www.npr.org/sections/goatsandsoda/2016/...,It was a hospital — but to psychologist Ink...


In [20]:
# remove publisher tags from article titles 

just_titles = []

for title in small_data.title:
  title = re.sub(r"(- Breitbart)(?!.*\1)", '', title)
  title = re.sub(r'(- The New York Times)(?!.*\1)', '', title)
  just_titles.append(title)

just_titles[:5]

['Chaos in the Family, Chaos in the State: The White Working Class’s Dysfunction',
 ' US Civil Rights Commission Will Observe Standing Rock\xa0Standoff',
 'Venezuela hunts rogue helicopter attackers, Maduro foes suspicious',
 'Fruit juice isn’t much better for you than soda. Let’s stop pretending otherwise.',
 'Sessions won’t testify at congressional budget hearings but at Senate intelligence hearing instead']

In [21]:
len(just_titles)

10000

In [22]:
# join with content of article

small_text = list(map(lambda i,j: i + " " + j, just_titles, small_data.content))

small_text[:5]

['Chaos in the Family, Chaos in the State: The White Working Class’s Dysfunction Michael Brendan Dougherty is bitter. I think that I can write that in both truth and charity. (I think you might even say that he and I are friends.) Dougherty is a conservative of the sort sometimes advertised as “paleo” and served as national correspondent for The American Conservative. Like many conservative writers with those associations, Dougherty spends a great deal of time lambasting the conservative movement and its organs, from which he feels, for whatever reason, estranged  —   an alienation that carries with it more than a little to suggest that it is somewhat personal. You know: Them. Donald Trump is the headline, and explaining the benighted white working class to Them is the main matter. Sanctimony is the literary mode, for Dougherty and for many others doing the same work with less literary facility. Never mind the petty sneering (as though the conservative movement were populated by septua

In [23]:
len(small_text)

10000

## Create Matrix of TF-IDF Features

In [0]:
def preprocess_text(text):

  # function to remove punctuation 
  def Punctuation(string): 
    return re.sub(r'[\W_]', ' ', string)

  # remove punctuation and perform tokenization
  text = Punctuation(text.lower()).split()

  # remove stop words and stem
  stop_words = set(stopwords.words('english'))
  stemmer = SnowballStemmer("english")
  text = [stemmer.stem(t) for t in text if not t in stop_words]

  return text


In [0]:
# tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', use_idf=True, tokenizer=preprocess_text, ngram_range=(1,3))

In [40]:
# fit just_titles to vectorizer
tfidf_matrix = tfidf_vectorizer.fit_transform(small_text)
print(tfidf_matrix.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(10000, 5000)


In [0]:
# reduce memory 
tfidf_matrix = tfidf_matrix.astype(np.float32)

In [42]:
tfidf_matrix

<10000x5000 sparse matrix of type '<class 'numpy.float32'>'
	with 2077663 stored elements in Compressed Sparse Row format>

## Create Matrix of Word2Vec Features

## Universal Encoder

In [24]:
# load the Universal Sentence Encoder's TF Hub module

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)

def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [0]:
# reduce logging output
logging.set_verbosity(logging.ERROR)

# compute a representation for each message
message_embeddings = embed(small_text)

In [0]:
# semantic similarity of two sentences can be trivially computed as the inner product of the encodings
corr = np.inner(message_embeddings, message_embeddings)

In [27]:
# data frame of titles and semantic similarities
corr_df = pd.DataFrame(corr)
corr_df.columns = just_titles
corr_df.index = just_titles

corr_df.head()

,"Chaos in the Family, Chaos in the State: The White Working Class’s Dysfunction",US Civil Rights Commission Will Observe Standing Rock Standoff,"Venezuela hunts rogue helicopter attackers, Maduro foes suspicious",Fruit juice isn’t much better for you than soda. Let’s stop pretending otherwise.,Sessions won’t testify at congressional budget hearings but at Senate intelligence hearing instead,Qatar: UAE and Saudi Arabia step up pressure in diplomatic crisis,’The 4-Hour Workweek’ author says a 3-step process he learned from Tony Robbins drastically improved his life,Rick Perry Attacked for Keeping His Word,Watch how a mathematician explains an astonishing coincidence,Second baby bald eagle begins hatching process at National Arboretum,In search of the Chibok girls and the meaning of their kidnapping,Biden Jokingly Thanks Trump for ’Making the American People Look in the Mirror’,"Hillary Clinton’s plan to undo the school-to-prison pipeline, explained","HIV Was Likely Transmitted On A Gay Porn Set, CDC Reports",Man survives knife in eye socket,Viacom in talks with Jim Gianopulos to head Paramount,Emmanuel Macron Declared Next French President,Ivanka Trump cuts off Cosmo interview after tough questioning,Hillary Campaign Denies Report of Campaign Shake-up After New Hampshire,VIDEO: Alabama High School Girl Fires Stun Gun at Teacher,How Sexism Held Back Space Exploration,Trump-Clinton Showdown Most-Watched Debate in History,How Republicans came to embrace anti-environmentalism,"Taliban storm German consulate in Afghan city, four killed",Lake Oroville dam: emergency staff race to fix spillway before more rain strikes,Viewsroom Predictions 2017: Part 2,Trump is finding it easier to tear down old policies than to build his own,Iran’s ex-president writes Obama on seized assets,U.S. charges Florida man in case linked to JPMorgan hacking probe,The real reason superrich people hate Obama,Trump Tower Lobby Evacuated by Police after Reports of Suspicious Package,Stormtrooper behind infamous ‘Star Wars’ blooper breaks his silence,Patient Diplomacy And A Reluctance To Act: Obama’s Mark On Foreign Policy,Millennials are totally mixed up about what they believe in,Cologne Police Attacked for ’Racial Profiling’ During NYE,"BIAS ALERT: CNN reporter says Hannity, Limbaugh want Hillary ’dying’",Jill Stein files for recount in Wisconsin,A little-known Pakistani tribe that loves wine and whiskey fears its Muslim neighbors,Valeant to appoint interim CEO as Pearson remains hospitalized: source,Wall Street rises as data points to accelerating economy,...,This week we saw that the Republican Party — not just Trump — is the problem,What the holidays are like for a recovering alcoholic like me,Doping has always been part of the Olympics. Of course Russia got off the hook.,Here’s how to not to get old lady hands,Texas Health Officials Brace for More Zika Virus Cases After Six Confirmed Statewide,Cruz: Trump Attacked My Wife Because He’s ’Scared’,Canada says most border-crossing asylum seekers were in U.S. legally,"LGBT employees protected from workplace discrimination, appeals court rules",Outgoing US ambassador: America is ’not an ethno-state’,We asked legal experts if Greg Gianforte can serve in Congress if he’s convicted of assault in reporter ’body-slam’ case,What next for Maria Sharapova as she prepares to appeal against her drug ban?,Donald Trump promises to deport 3 million “illegal immigrant criminals.” That’s literally impossible.,American democracy is winning... so far,USSS chief says ’No friction’ with Trump’s private security,The simple arithmetic that could jump-start America’s economic growth,Trump and Clinton join mourners at 9/11 anniversary ceremony,"Lucille Ball statue needs a makeover, locals say",FBI monitored former Trump campaign adviser Carter Page on Russia,Dr. Jane Orient: ‘Universal Coverage Means Less Care’,"Pro-Trump Art Show Finds New Venue, Despite Cancellation Attempt And Legal Threats",Marvel Comics Cancels Black Lives Matter-

In [28]:
# find index of 5 most similar titles 
top5_ind = [] 
for i in range(len(corr_df)):
  top5_ind.append(list(list(zip(*heapq.nlargest(5, enumerate(corr_df.iloc[i,:]), key=operator.itemgetter(1))))[0]))

# show most similar titles -- sanity check
for ind in top5_ind:
  print([just_titles[i] for i in ind])

Streaming output truncated to the last 5000 lines.
['Cretaceous Park: Discovery of horse-sized dino could solve T. rex mystery', 'Newly Discovered Dinosaur Helps Explain Rise Of Tyrannosaurs', 'Scientists discover dinosaur trapped in amber in unprecedented find', 'Scientists reveal most accurate depiction of a dinosaur ever created', 'How scientists actually could bring dinosaurs back to\xa0life']
['Donald Trump says he can’t release his tax records because of an audit. Mitt Romney calls bullshit.', 'Why political elites ‘tax-shame’ outsiders like\xa0Trump', 'What Trump tax report could mean for his campaign', 'New York Times: Trump Losses May Mean He Didn’t Pay Taxes for Years ', 'Christie defends Trump’s leaked tax return, says nothing illegal, ’no apologies’']
['Sorry, ‘The Girl on the Train,’ you’re no ‘Gone\xa0Girl’', 'The Accountant and The Girl on the Train are obsessed with twists, at the expense of being good movies', 'The Invitation: A Dinner Party of Nightmares', '‘The Arran

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
np.array(message_embeddings).shape

(10000, 512)

In [62]:
np.array(message_embeddings[1]).reshape(1, -1).shape

(1, 512)

In [61]:
 km.cluster_centers_[1].reshape(1, -1).shape


(1, 512)

## Run KMeans Clustering 

In [0]:
num_clusters = 15

km = KMeans(n_clusters=num_clusters, random_state=10)

#km.fit(tfidf_matrix)
km.fit(message_embeddings)

clusters = km.labels_.tolist()

In [0]:
# cosine similarities for each row with cluster center
#cos_sim = [cosine_similarity(tfidf_matrix[i].toarray(), km.cluster_centers_[label].reshape(1, -1))[0][0] for i,label in enumerate(km.labels_)]
cos_sim = [cosine_similarity(np.array(message_embeddings[i]).reshape(1, -1), km.cluster_centers_[label].reshape(1, -1))[0][0] for i,label in enumerate(km.labels_)]

In [65]:
# sum of squared distances of samples to their closest cluster center.
km.inertia_

5976.191444143563

In [0]:
# simple dataframe with title, cluster, and cosine similarity 
title_data = pd.DataFrame({'title':small_data.title, 'cluster':clusters, 'cos_sim':cos_sim})

In [67]:
title_data

,title,cluster,cos_sim
0,"Chaos in the Family, Chaos in the State: The W...",4,0.668791
1,US Civil Rights Commission Will Observe Stand...,0,0.650066
2,"Venezuela hunts rogue helicopter attackers, Ma...",11,0.702218
3,Fruit juice isn’t much better for you than sod...,6,0.391130
4,Sessions won’t testify at congressional budget...,7,0.751112
...,...,...,...
9995,Patient secretly recorded doctors as they oper...,5,0.518504
9996,Fox News Poll: Clinton edges Trump by two poin...,1,0.714751
9997,The Atlantic Politics & Policy Daily: Trump’s...,10,0.788887
9998,A Son In Chains. A Depressed Mom. Here’s What ...,6,0.569110


In [68]:
# inspect titles with highest cosine similarity in clusters 
top_5 = title_data.groupby('cluster')['cos_sim'].nlargest(5)
for i,ind in top_5.index:
  print("cluster", i)
  print(just_titles[ind])
  print('-------------')

cluster 0
L.A. Sheriff Faces Protests for Opposition to ’Sanctuary State’ 
-------------
cluster 0
Texas governor signs bill banning sanctuary cities 
-------------
cluster 0
Ban on Arizona Sheriff Arpaio’s workplace raids lifted: court
-------------
cluster 0
Court Temporarily Blocks Trump’s Travel Ban, and Airlines Are Told to Allow Passengers 
-------------
cluster 0
Arrested Mexican ’dreamer’ immigrant seeks new U.S. legal channel
-------------
cluster 1
Toned-down Trump does not mean retreat from policy: aide
-------------
cluster 1
Karl Rove: Key questions this week for Trump, Hillary
-------------
cluster 1
The GOP now belongs to Trump. What are Republicans going to do about it?
-------------
cluster 1
Trump In Triumph: The Man Who Wasn’t Welcome In The Game Takes Home The Prize
-------------
cluster 1
Clinton sets sights on Trump, general election after huge win in South Carolina
-------------
cluster 2
Islamic State claims attack on Pakistan police academy, 59 dead
-----------

## Predict Input

In [0]:
input_topic = ["Hillary Clinton defends handling of Benghazi attack", "Women's March Highlights", "Hillary Clinton emails"]

In [0]:
# run tfidf on input 
tfidf_input = tfidf_vectorizer.transform(input_topic)

# make list of tfidf_input
tfidf_input = [tfidf_input.getrow(i).toarray()[0].tolist() for i in range(tfidf_input.shape[0])]

In [0]:
# run tfidf on input 
tfidf_input = tfidf_vectorizer.transform(input_topic)

# make list of tfidf_input
tfidf_input = [tfidf_input.getrow(i).toarray()[0].tolist() for i in range(tfidf_input.shape[0])]

In [0]:
# make list of tfidf_matrix
tfidf_list = [tfidf_matrix.getrow(i).toarray()[0].tolist() for i in range(tfidf_matrix.shape[0])]


In [0]:
np.random.seed(5)
X_train = tfidf_list
y_train = title_data.cluster
X_test  = tfidf_input

# Create and fit a nearest-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train) 

res = knn.predict(X_test)

In [0]:
print(res)

[10  6 10]


In [0]:
for i,ind in top_5.index:
  if (i==10) | (i==6):
    print("cluster", i)
    print(just_titles[ind])
    print('-------------')

cluster 6
50 Wonderful Things From 2016
-------------
cluster 6
We read all 20 National Book Award nominees for 2016. Here’s what we thought.
-------------
cluster 6
In A Genre Crowded With Bros, Cam Lifts Off
-------------
cluster 6
Marie Kondo and the Ruthless War on Stuff 
-------------
cluster 6
Being Aaron Burr
-------------
cluster 10
From Whitewater to Benghazi: A Clinton-Scandal Primer
-------------
cluster 10
Hillary, Not Trump, Forced Us to Revisit Bill Clinton’s Scandals
-------------
cluster 10
Can we please be really truly done with Hillary?
-------------
cluster 10
A new tell-all about the Clinton campaign is a searing indictment of the candidate herself
-------------
cluster 10
Clinton: ’People should and do trust me’
-------------
